# Fetching model, testing and exporting it

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import requests
from PIL import Image
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
original_image_cache = {}

def preprocess_image(image):
  image = np.array(image)
  # reshape into shape [batch_size, height, width, num_channels]
  img_reshaped = tf.reshape(image, [1, image.shape[0], image.shape[1], image.shape[2]])
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  image = tf.image.convert_image_dtype(img_reshaped, tf.float32)
  return image

def load_image_from_url(img_url):
  """Returns an image with shape [1, height, width, num_channels]."""
  user_agent = {'User-agent': 'Colab Sample (https://tensorflow.org)'}
  response = requests.get(img_url, headers=user_agent)
  image = Image.open(BytesIO(response.content))
  image = preprocess_image(image)
  return image

def load_image_from_file(image_file):
  image = Image.open(image_file)
  image = preprocess_image(image)
  return image

img_rm = load_image_from_url('https://upload.wikimedia.org/wikipedia/commons/f/f7/Honey_comb.jpg')
img_file = load_image_from_file('Honey_comb.jpg')

def load_image(image_url, image_size=256, dynamic_size=False, max_dynamic_size=512):
  """Loads and preprocesses images."""
  # Cache image file locally.
  if image_url in original_image_cache:
    img = original_image_cache[image_url]
  elif image_url.startswith('https://'):
    img = load_image_from_url(image_url)
  else:
    fd = tf.io.gfile.GFile(image_url, 'rb')
    img = preprocess_image(Image.open(fd))
  original_image_cache[image_url] = img
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img_raw = img
  if tf.reduce_max(img) > 1.0:
    img = img / 255.
  if len(img.shape) == 3:
    img = tf.stack([img, img, img], axis=-1)
  if not dynamic_size:
    img = tf.image.resize_with_pad(img, image_size, image_size)
  elif img.shape[1] > max_dynamic_size or img.shape[2] > max_dynamic_size:
    img = tf.image.resize_with_pad(img, max_dynamic_size, max_dynamic_size)
  return img, img_raw

# TODO: my new function
def load_images_into_batch(image_urls, image_size=256, dynamic_size=False, max_dynamic_size=512):
  img_batch = None
  
  for image_url in image_urls:
    if img_batch is None:
      # handling first image in the batch
      img_batch, _ = load_image(image_url, image_size=256, dynamic_size=False, max_dynamic_size=512)
      continue
    # 2nd and onward images in the batch
    img, _ = load_image(image_url, image_size=256, dynamic_size=False, max_dynamic_size=512)
    img_batch = tf.concat([img_batch, img], 0)
  
  return img_batch

def show_image(image, title=''):
  image_size = image.shape[1]
  w = (image_size * 6) // 320
  plt.figure(figsize=(w, w))
  plt.imshow(image[0], aspect='equal')
  plt.axis('off')
  plt.title(title)
  plt.show()

In [ ]:
image_size = 224
dynamic_size = False

model_name = "efficientnetv2-s" # @param ['efficientnetv2-s', 'efficientnetv2-m', 'efficientnetv2-l', 'efficientnetv2-s-21k', 'efficientnetv2-m-21k', 'efficientnetv2-l-21k', 'efficientnetv2-xl-21k', 'efficientnetv2-b0-21k', 'efficientnetv2-b1-21k', 'efficientnetv2-b2-21k', 'efficientnetv2-b3-21k', 'efficientnetv2-s-21k-ft1k', 'efficientnetv2-m-21k-ft1k', 'efficientnetv2-l-21k-ft1k', 'efficientnetv2-xl-21k-ft1k', 'efficientnetv2-b0-21k-ft1k', 'efficientnetv2-b1-21k-ft1k', 'efficientnetv2-b2-21k-ft1k', 'efficientnetv2-b3-21k-ft1k', 'efficientnetv2-b0', 'efficientnetv2-b1', 'efficientnetv2-b2', 'efficientnetv2-b3', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'bit_s-r50x1', 'inception_v3', 'inception_resnet_v2', 'resnet_v1_50', 'resnet_v1_101', 'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152', 'nasnet_large', 'nasnet_mobile', 'pnasnet_large', 'mobilenet_v2_100_224', 'mobilenet_v2_130_224', 'mobilenet_v2_140_224', 'mobilenet_v3_small_100_224', 'mobilenet_v3_small_075_224', 'mobilenet_v3_large_100_224', 'mobilenet_v3_large_075_224']

model_handle_map = {
  "efficientnetv2-s": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/classification/2",
  "efficientnetv2-m": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_m/classification/2",
  "efficientnetv2-l": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_l/classification/2",
  "efficientnetv2-s-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/classification/2",
  "efficientnetv2-m-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/classification/2",
  "efficientnetv2-l-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/classification/2",
  "efficientnetv2-xl-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/classification/2",
  "efficientnetv2-b0-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/classification/2",
  "efficientnetv2-b1-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b1/classification/2",
  "efficientnetv2-b2-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b2/classification/2",
  "efficientnetv2-b3-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b3/classification/2",
  "efficientnetv2-s-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_s/classification/2",
  "efficientnetv2-m-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_m/classification/2",
  "efficientnetv2-l-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_l/classification/2",
  "efficientnetv2-xl-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/classification/2",
  "efficientnetv2-b0-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/classification/2",
  "efficientnetv2-b1-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b1/classification/2",
  "efficientnetv2-b2-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b2/classification/2",
  "efficientnetv2-b3-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/classification/2",
  "efficientnetv2-b0": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/classification/2",
  "efficientnetv2-b1": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b1/classification/2",
  "efficientnetv2-b2": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b2/classification/2",
  "efficientnetv2-b3": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b3/classification/2",
  "efficientnet_b0": "https://tfhub.dev/tensorflow/efficientnet/b0/classification/1",
  "efficientnet_b1": "https://tfhub.dev/tensorflow/efficientnet/b1/classification/1",
  "efficientnet_b2": "https://tfhub.dev/tensorflow/efficientnet/b2/classification/1",
  "efficientnet_b3": "https://tfhub.dev/tensorflow/efficientnet/b3/classification/1",
  "efficientnet_b4": "https://tfhub.dev/tensorflow/efficientnet/b4/classification/1",
  "efficientnet_b5": "https://tfhub.dev/tensorflow/efficientnet/b5/classification/1",
  "efficientnet_b6": "https://tfhub.dev/tensorflow/efficientnet/b6/classification/1",
  "efficientnet_b7": "https://tfhub.dev/tensorflow/efficientnet/b7/classification/1",
  "bit_s-r50x1": "https://tfhub.dev/google/bit/s-r50x1/ilsvrc2012_classification/1",
  "inception_v3": "https://tfhub.dev/google/imagenet/inception_v3/classification/4",
  "inception_resnet_v2": "https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/4",
  "resnet_v1_50": "https://tfhub.dev/google/imagenet/resnet_v1_50/classification/4",
  "resnet_v1_101": "https://tfhub.dev/google/imagenet/resnet_v1_101/classification/4",
  "resnet_v1_152": "https://tfhub.dev/google/imagenet/resnet_v1_152/classification/4",
  "resnet_v2_50": "https://tfhub.dev/google/imagenet/resnet_v2_50/classification/4",
  "resnet_v2_101": "https://tfhub.dev/google/imagenet/resnet_v2_101/classification/4",
  "resnet_v2_152": "https://tfhub.dev/google/imagenet/resnet_v2_152/classification/4",
  "nasnet_large": "https://tfhub.dev/google/imagenet/nasnet_large/classification/4",
  "nasnet_mobile": "https://tfhub.dev/google/imagenet/nasnet_mobile/classification/4",
  "pnasnet_large": "https://tfhub.dev/google/imagenet/pnasnet_large/classification/4",
  "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/4",
  "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4",
  "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/classification/4",
  "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/classification/5",
  "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/classification/5",
  "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/classification/5",
  "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/classification/5",
}

model_image_size_map = {
  "efficientnetv2-s": 384,
  "efficientnetv2-m": 480,
  "efficientnetv2-l": 480,
  "efficientnetv2-b0": 224,
  "efficientnetv2-b1": 240,
  "efficientnetv2-b2": 260,
  "efficientnetv2-b3": 300,
  "efficientnetv2-s-21k": 384,
  "efficientnetv2-m-21k": 480,
  "efficientnetv2-l-21k": 480,
  "efficientnetv2-xl-21k": 512,
  "efficientnetv2-b0-21k": 224,
  "efficientnetv2-b1-21k": 240,
  "efficientnetv2-b2-21k": 260,
  "efficientnetv2-b3-21k": 300,
  "efficientnetv2-s-21k-ft1k": 384,
  "efficientnetv2-m-21k-ft1k": 480,
  "efficientnetv2-l-21k-ft1k": 480,
  "efficientnetv2-xl-21k-ft1k": 512,
  "efficientnetv2-b0-21k-ft1k": 224,
  "efficientnetv2-b1-21k-ft1k": 240,
  "efficientnetv2-b2-21k-ft1k": 260,
  "efficientnetv2-b3-21k-ft1k": 300, 
  "efficientnet_b0": 224,
  "efficientnet_b1": 240,
  "efficientnet_b2": 260,
  "efficientnet_b3": 300,
  "efficientnet_b4": 380,
  "efficientnet_b5": 456,
  "efficientnet_b6": 528,
  "efficientnet_b7": 600,
  "inception_v3": 299,
  "inception_resnet_v2": 299,
  "mobilenet_v2_100_224": 224,
  "mobilenet_v2_130_224": 224,
  "mobilenet_v2_140_224": 224,
  "nasnet_large": 331,
  "nasnet_mobile": 224,
  "pnasnet_large": 331,
  "resnet_v1_50": 224,
  "resnet_v1_101": 224,
  "resnet_v1_152": 224,
  "resnet_v2_50": 224,
  "resnet_v2_101": 224,
  "resnet_v2_152": 224,
  "mobilenet_v3_small_100_224": 224,
  "mobilenet_v3_small_075_224": 224,
  "mobilenet_v3_large_100_224": 224,
  "mobilenet_v3_large_075_224": 224,
}

model_handle = model_handle_map[model_name]

print(f"Selected model: {model_name} : {model_handle}")


max_dynamic_size = 512
if model_name in model_image_size_map:
  image_size = model_image_size_map[model_name]
  dynamic_size = False
  print(f"Images will be converted to {image_size}x{image_size}")
else:
  dynamic_size = True
  print(f"Images will be capped to a max size of {max_dynamic_size}x{max_dynamic_size}")

labels_file = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"

#download labels and creates a maps
downloaded_file = tf.keras.utils.get_file("labels.txt", origin=labels_file)

classes = []

with open(downloaded_file) as f:
  labels = f.readlines()
  classes = [l.strip() for l in labels]

In [ ]:
image_name = "tiger" # @param ['tiger', 'bus', 'car', 'cat', 'dog', 'apple', 'banana', 'turtle', 'flamingo', 'piano', 'honeycomb', 'teapot']

images_for_test_map = {
    "tiger": "https://upload.wikimedia.org/wikipedia/commons/b/b0/Bengal_tiger_%28Panthera_tigris_tigris%29_female_3_crop.jpg",
    #by Charles James Sharp, CC BY-SA 4.0 <https://creativecommons.org/licenses/by-sa/4.0>, via Wikimedia Commons
    "bus": "https://upload.wikimedia.org/wikipedia/commons/6/63/LT_471_%28LTZ_1471%29_Arriva_London_New_Routemaster_%2819522859218%29.jpg",
    #by Martin49 from London, England, CC BY 2.0 <https://creativecommons.org/licenses/by/2.0>, via Wikimedia Commons
    "car": "https://upload.wikimedia.org/wikipedia/commons/4/49/2013-2016_Toyota_Corolla_%28ZRE172R%29_SX_sedan_%282018-09-17%29_01.jpg",
    #by EurovisionNim, CC BY-SA 4.0 <https://creativecommons.org/licenses/by-sa/4.0>, via Wikimedia Commons
    "cat": "https://upload.wikimedia.org/wikipedia/commons/4/4d/Cat_November_2010-1a.jpg",
    #by Alvesgaspar, CC BY-SA 3.0 <https://creativecommons.org/licenses/by-sa/3.0>, via Wikimedia Commons
    "dog": "https://upload.wikimedia.org/wikipedia/commons/archive/a/a9/20090914031557%21Saluki_dog_breed.jpg",
    #by Craig Pemberton, CC BY-SA 3.0 <https://creativecommons.org/licenses/by-sa/3.0>, via Wikimedia Commons
    "apple": "https://upload.wikimedia.org/wikipedia/commons/1/15/Red_Apple.jpg",
    #by Abhijit Tembhekar from Mumbai, India, CC BY 2.0 <https://creativecommons.org/licenses/by/2.0>, via Wikimedia Commons
    "banana": "https://upload.wikimedia.org/wikipedia/commons/1/1c/Bananas_white_background.jpg",
    #by fir0002  flagstaffotos [at] gmail.com       Canon 20D + Tamron 28-75mm f/2.8, GFDL 1.2 <http://www.gnu.org/licenses/old-licenses/fdl-1.2.html>, via Wikimedia Commons
    "turtle": "https://upload.wikimedia.org/wikipedia/commons/8/80/Turtle_golfina_escobilla_oaxaca_mexico_claudio_giovenzana_2010.jpg",
    #by Claudio Giovenzana, CC BY-SA 3.0 <https://creativecommons.org/licenses/by-sa/3.0>, via Wikimedia Commons
    "flamingo": "https://upload.wikimedia.org/wikipedia/commons/b/b8/James_Flamingos_MC.jpg",
    #by Christian Mehlführer, User:Chmehl, CC BY 3.0 <https://creativecommons.org/licenses/by/3.0>, via Wikimedia Commons
    "piano": "https://upload.wikimedia.org/wikipedia/commons/d/da/Steinway_%26_Sons_upright_piano%2C_model_K-132%2C_manufactured_at_Steinway%27s_factory_in_Hamburg%2C_Germany.png",
    #by "Photo: © Copyright Steinway & Sons", CC BY-SA 3.0 <https://creativecommons.org/licenses/by-sa/3.0>, via Wikimedia Commons
    "honeycomb": "https://upload.wikimedia.org/wikipedia/commons/f/f7/Honey_comb.jpg",
    #by Merdal, CC BY-SA 3.0 <http://creativecommons.org/licenses/by-sa/3.0/>, via Wikimedia Commons
    "teapot": "https://upload.wikimedia.org/wikipedia/commons/4/44/Black_tea_pot_cropped.jpg",
    #by Mendhak, CC BY-SA 2.0 <https://creativecommons.org/licenses/by-sa/2.0>, via Wikimedia Commons
}

img_url = images_for_test_map[image_name]
image, original_image = load_image(img_url, image_size, dynamic_size, max_dynamic_size)
show_image(image, 'Scaled image')

In [ ]:
import sys
print(sys.getsizeof(image.numpy()))
print(type(image))
print(sys.getsizeof(original_image))
print(sys.getsizeof(load_image_from_file('Honey_comb.jpg').numpy()))

In [ ]:
classifier = hub.load(model_handle)

input_shape = image.shape
warmup_input = tf.random.uniform(input_shape, 0, 1.0)
%time warmup_logits = classifier(warmup_input).numpy()

In [ ]:
batched_images.shape

In [ ]:
# Run model on image
%time probabilities = tf.nn.softmax(classifier(image)).numpy()

top_5 = tf.argsort(probabilities, axis=-1, direction="DESCENDING")[0][:5].numpy()
np_classes = np.array(classes)

# Some models include an additional 'background' class in the predictions, so
# we must account for this when reading the class labels.
includes_background_class = probabilities.shape[1] == 1001

for i, item in enumerate(top_5):
  class_index = item if includes_background_class else item + 1
  line = f'({i+1}) {class_index:4} - {classes[class_index]}: {probabilities[0][top_5][i]}'
  print(line)

show_image(image, '')

In [ ]:
img_urls = [img_url, images_for_test_map["honeycomb"], images_for_test_map["teapot"]]

batched_images = load_images_into_batch(img_urls, image_size, dynamic_size, max_dynamic_size)

In [ ]:
%time batch_probabilities = tf.nn.softmax(classifier(batched_images)).numpy()

## Run model on image
#%time probabilities = tf.nn.softmax(classifier(image)).numpy()

batch_top_5 = []
batch_size = batched_images.shape[0]
np_classes = np.array(classes)

for batch_ix in range(batch_size):
    top_5 = tf.argsort(batch_probabilities, axis=-1, direction="DESCENDING")[batch_ix][:5].numpy()

    print(top_5)

    # Some models include an additional 'background' class in the predictions, so
    # we must account for this when reading the class labels.
    includes_background_class = batch_probabilities.shape[1] == 1001

    for i, item in enumerate(top_5):
      print(i)

      class_index = item if includes_background_class else item + 1
      line = f'({i+1}) {class_index:4} - {classes[class_index]}: {batch_probabilities[batch_ix][top_5][i]}'
      print(line)
    show_image(tf.expand_dims(batched_images[batch_ix], 0), '')



In [ ]:
import os

model_dir = f"./{model_name}"
os.makedirs(model_dir, exist_ok=False)
tf.saved_model.save(classifier, model_dir)

# Create Inference script

In [ ]:
! pygmentize model/code/inference.py

# Packaging model artifacts

In [42]:
import sagemaker
from sagemaker.tensorflow import TensorFlowModel


import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
#role = get_execution_role() 
role="arn:aws:iam::941656036254:role/service-role/AmazonSageMaker-ExecutionRole-20210904T193230" # TODO: this has to be replaced

bucket = sagemaker_session.default_bucket()
prefix = 'tf-serving'
s3_path = 's3://{}/{}'.format(bucket, prefix)


In [ ]:
!tar -C "$PWD" -czf model_failed.tar.gz  model/

In [ ]:

model_data = sagemaker_session.upload_data('model_failed.tar.gz',
                                           bucket,
                                           os.path.join(prefix, 'model'))

In [43]:
env = {"SAGEMAKER_GUNICORN_LOGLEVEL":"debug",
       "SAGEMAKER_TFS_ENABLE_BATCHING":"true",
       "SAGEMAKER_TFS_MAX_BATCH_SIZE":"4",
       "SAGEMAKER_TFS_BATCH_TIMEOUT_MICROS":"100000",
       "SAGEMAKER_TFS_NUM_BATCH_THREADS":"6",
       "SAGEMAKER_TFS_MAX_ENQUEUED_BATCHES":"6",
       "USE_GRPC":"true"
       }

# The "Model" object doesn't create a SageMaker Model until a Transform Job or Endpoint is created.
tensorflow_serving_model = TensorFlowModel(model_data=model_data,
                                 name="efficientnetv2-1",
                                 role=role,
                                 framework_version='2.8',
                                 env=env,
                                 sagemaker_session=sagemaker_session)

In [44]:
predictor = tensorflow_serving_model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Using already existing model: efficientnetv2-1


----------!

In [47]:
response_remote = predictor.predict(image.numpy())

In [48]:
probabilities = np.array(response_remote['predictions'])

top_5 = tf.argsort(probabilities, axis=-1, direction="DESCENDING")[0][:5].numpy()
np_classes = np.array(classes)

# Some models include an additional 'background' class in the predictions, so
# we must account for this when reading the class labels.
includes_background_class = probabilities.shape[1] == 1001

for i, item in enumerate(top_5):
  class_index = item if includes_background_class else item + 1
  line = f'({i+1}) {class_index:4} - {classes[class_index]}: {probabilities[0][top_5][i]}'
  print(line)


(1)  293 - tiger: 9.22827721
(2)  283 - tiger cat: 7.63697577
(3)  291 - jaguar: 3.41268444
(4)  282 - tabby: 2.57228494
(5)  288 - lynx: 2.52746487


# Local mode testing

In [1]:
import boto3
from sagemaker.local import LocalSession

sagemaker_local_session = LocalSession()
sagemaker_local_session.config = {'local': {'local_code': True}}
account = boto3.client('sts').get_caller_identity().get('Account')
role = "arn:aws:iam::941656036254:role/service-role/AmazonSageMaker-ExecutionRole-20210904T193230"

TypeError: load() missing 1 required positional argument: 'Loader'

In [ ]:
import os
import subprocess

instance_type = "local"

try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
except:
    pass

print("Instance type = " + instance_type)

In [ ]:
model_data = "s3://sagemaker-us-east-1-941656036254/tf-serving/model/model_failed.tar.gz"

In [69]:
import sagemaker
from sagemaker.tensorflow import TensorFlowModel

env = {"SAGEMAKER_GUNICORN_LOGLEVEL":"debug",
       "SAGEMAKER_TFS_ENABLE_BATCHING":"true",
       "SAGEMAKER_TFS_MAX_BATCH_SIZE":"4",
       "SAGEMAKER_TFS_BATCH_TIMEOUT_MICROS":"100000",
       "SAGEMAKER_TFS_NUM_BATCH_THREADS":"6",
       "SAGEMAKER_TFS_MAX_ENQUEUED_BATCHES":"6",
       "USE_GRPC":"true",
       }


# The "Model" object doesn't create a SageMaker Model until a Transform Job or Endpoint is created.
local_tensorflow_serving_model = TensorFlowModel(model_data=model_data,
                                 name="efficientnetv2-1",
                                 image_uri="tf_serving:extended",
                                 entry_point="inference.py",
                                 source_dir="model/code",
                                 role=role,
                                 framework_version='2.8',
                                 env=env,
                                 sagemaker_session=sagemaker_local_session)

In [70]:
local_predictor = local_tensorflow_serving_model.deploy(initial_instance_count=1, instance_type=instance_type)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Attaching to 30hwpfjzfr-algo-1-hdzgd
30hwpfjzfr-algo-1-hdzgd | INFO:__main__:PYTHON SERVICE: False
30hwpfjzfr-algo-1-hdzgd | INFO:__main__:starting services
30hwpfjzfr-algo-1-hdzgd | INFO:__main__:batching is enabled
30hwpfjzfr-algo-1-hdzgd | INFO:tfs_utils:batching config: 
30hwpfjzfr-algo-1-hdzgd | max_batch_size { value: 4 }
30hwpfjzfr-algo-1-hdzgd | batch_timeout_micros { value: 100000 }
30hwpfjzfr-algo-1-hdzgd | num_batch_threads { value: 6 }
30hwpfjzfr-algo-1-hdzgd | max_enqueued_batches { value: 6 }
30hwpfjzfr-algo-1-hdzgd | 
30hwpfjzfr-algo-1-hdzgd | 
30hwpfjzfr-algo-1-hdzgd | INFO:__main__:multi-model endpoint is enabled, TFS model servers will be started later
30hwpfjzfr-algo-1-hdzgd | INFO:__main__:nginx config: 
30hwpfjzfr-algo-1-hdzgd | load_module modules/ngx_http_js_module.so;
30hwpfjzfr-algo-1-hdzgd | 
30hwpfjzfr-algo-1-hdzgd | worker_processes auto;
30hwpfjzfr-algo-1-hdzgd | daemon off;
30hwpfjzfr-algo-1-hdzgd | pid /tmp/nginx.pid;
30hwpfjzfr-algo-1-hdzgd | error_log  

RuntimeError: Giving up, endpoint didn't launch correctly

30hwpfjzfr-algo-1-hdzgd | INFO:__main__:gunicorn version info:
30hwpfjzfr-algo-1-hdzgd | gunicorn (version 20.1.0)
30hwpfjzfr-algo-1-hdzgd | INFO:__main__:started gunicorn (pid: 270)
30hwpfjzfr-algo-1-hdzgd | [2022-08-07 17:01:16 +0000] [270] [DEBUG] Current configuration:
30hwpfjzfr-algo-1-hdzgd |   config: ./gunicorn.conf.py
30hwpfjzfr-algo-1-hdzgd |   wsgi_app: None
30hwpfjzfr-algo-1-hdzgd |   bind: ['unix:/tmp/gunicorn.sock']
30hwpfjzfr-algo-1-hdzgd |   backlog: 2048
30hwpfjzfr-algo-1-hdzgd |   workers: 1
30hwpfjzfr-algo-1-hdzgd |   worker_class: gevent
30hwpfjzfr-algo-1-hdzgd |   threads: 1
30hwpfjzfr-algo-1-hdzgd |   worker_connections: 1000
30hwpfjzfr-algo-1-hdzgd |   max_requests: 0
30hwpfjzfr-algo-1-hdzgd |   max_requests_jitter: 0
30hwpfjzfr-algo-1-hdzgd |   timeout: 30
30hwpfjzfr-algo-1-hdzgd |   graceful_timeout: 30
30hwpfjzfr-algo-1-hdzgd |   keepalive: 2
30hwpfjzfr-algo-1-hdzgd |   limit_request_line: 4094
30hwpfjzfr-algo-1-hdzgd |   limit_request_fields: 100
30hwpfjzfr-a

Exception in thread Thread-24:
Traceback (most recent call last):
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 852, in run
    _stream_output(self.process)
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 914, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 137

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 857, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/private/var/folders/zm/7yczzhb529xff1v0_ps_8k4h0000gp/T/tmpnvcck4mn/docker-compose.y

In [ ]:
response = local_predictor.predict(image.numpy())

In [ ]:
probabilities = np.array(response['predictions'])
print(probabilities)

In [ ]:
probabilities = np.array(response['predictions'])

top_5 = tf.argsort(probabilities, axis=-1, direction="DESCENDING")[0][:5].numpy()
np_classes = np.array(classes)

# Some models include an additional 'background' class in the predictions, so
# we must account for this when reading the class labels.
includes_background_class = probabilities.shape[1] == 1001

for i, item in enumerate(top_5):
  class_index = item if includes_background_class else item + 1
  line = f'({i+1}) {class_index:4} - {classes[class_index]}: {probabilities[0][top_5][i]}'
  print(line)


# Testing parallel executions

In [64]:
from multiprocessing import Pool, Value
import random
import time

NUM_PARALLEL_WORKERS = 4

response = local_predictor.predict(image.numpy())
possible_images = ['tiger', 'bus', 'car', 'cat', 'dog', 'apple', 'banana', 'turtle', 'flamingo', 'piano', 'honeycomb', 'teapot']


def predict_f(predictor):
    random_ix = random.randint(0, (len(possible_images)-1))
    img_url = images_for_test_map[possible_images[random_ix]]
    image, _ = load_image(img_url, image_size, dynamic_size, max_dynamic_size)
    predictor.predict(image.numpy())
    print(".")

##start_time = time.clock()
#p = Pool(NUM_PARALLEL_WORKERS)
#p.apply(predict_f, local_predictor)

for i in range(100):
    predict_f(local_predictor)


MaxRetryError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /invocations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fec92b77d90>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [61]:
from multiprocessing import Pool
import random
import time
import boto3

NUM_PARALLEL_WORKERS = 4

response = local_predictor.predict(image.numpy())
possible_images = ['tiger', 'bus', 'car', 'cat', 'dog', 'apple', 'banana', 'turtle', 'flamingo', 'piano', 'honeycomb', 'teapot']


def predict_f(predictor):
    random_ix = random.randint(0, (len(possible_images)-1))
    img_url = images_for_test_map[possible_images[random_ix]]
    image, _ = load_image(img_url, image_size, dynamic_size, max_dynamic_size)
    predictor.predict(image.numpy())
    print(".")

##start_time = time.clock()
#p = Pool(NUM_PARALLEL_WORKERS)
#p.apply(predict_f, local_predictor)

import numpy as np

stats = np.array([timer(predict_f, local_predictor) for _ in range(10)])
print(stats)


jfdvl7p9a5-algo-1-s1c1k | [2022-08-06 15:30:44 +0000] [155] [DEBUG] POST /invocations
jfdvl7p9a5-algo-1-s1c1k | INFO:tfs_utils:sagemaker tfs attributes: 
jfdvl7p9a5-algo-1-s1c1k | {}
jfdvl7p9a5-algo-1-s1c1k | output shape: [1, 1000]
jfdvl7p9a5-algo-1-s1c1k | [2022-08-06 15:30:45 +0000] [155] [DEBUG] Closing connection. 
jfdvl7p9a5-algo-1-s1c1k | 172.18.0.1 - - [06/Aug/2022:15:30:45 +0000] "POST /invocations HTTP/1.1" 200 20937 "-" "python-urllib3/1.26.8"
jfdvl7p9a5-algo-1-s1c1k | [2022-08-06 15:30:47 +0000] [155] [DEBUG] POST /invocations
jfdvl7p9a5-algo-1-s1c1k | INFO:tfs_utils:sagemaker tfs attributes: 
jfdvl7p9a5-algo-1-s1c1k | {}
jfdvl7p9a5-algo-1-s1c1k | output shape: [1, 1000]
jfdvl7p9a5-algo-1-s1c1k | [2022-08-06 15:30:48 +0000] [155] [DEBUG] Closing connection. 
.
jfdvl7p9a5-algo-1-s1c1k | 172.18.0.1 - - [06/Aug/2022:15:30:48 +0000] "POST /invocations HTTP/1.1" 200 20885 "-" "python-urllib3/1.26.8"
jfdvl7p9a5-algo-1-s1c1k | [2022-08-06 15:30:49 +0000] [155] [DEBUG] POST /invoca

In [58]:
from time import perf_counter

def timer(f,*args):   
    
    start = perf_counter()
    f(*args)
    return (1000 * (perf_counter() - start))

In [63]:
stats
print("\nPredictions for TF2 serving with gRPC : \n")
print('\nP95: ' + str(np.percentile(stats, 95)) + ' ms\n')    
print('P90: ' + str(np.percentile(stats, 90)) + ' ms\n')
print('Average: ' + str(np.average(stats)) + ' ms\n')


Predictions for TF2 serving with gRPC : 


P95: 2866.7889760003163 ms

P90: 2792.0670430001337 ms

Average: 2512.3298259000876 ms

jfdvl7p9a5-algo-1-s1c1k | [2022-08-06 16:38:30 +0000] [99] [CRITICAL] WORKER TIMEOUT (pid:155)
jfdvl7p9a5-algo-1-s1c1k | [2022-08-06 16:38:30 +0000] [155] [INFO] Worker exiting (pid: 155)
jfdvl7p9a5-algo-1-s1c1k | [2022-08-06 16:38:31 +0000] [99] [WARNING] Worker with pid 155 was terminated due to signal 9
jfdvl7p9a5-algo-1-s1c1k | [2022-08-06 16:38:31 +0000] [160] [INFO] Booting worker with pid: 160
jfdvl7p9a5-algo-1-s1c1k | /usr/local/lib/python3.9/importlib/__init__.py:127: DeprecatedWarning: Call to deprecated function __init__(...). API class may be removed in a future release, use falcon.App instead.
jfdvl7p9a5-algo-1-s1c1k |   return _bootstrap._gcd_import(name[level:], package, level)
jfdvl7p9a5-algo-1-s1c1k | INFO:python_service:Creating grpc channel for port: 9000
jfdvl7p9a5-algo-1-s1c1k | 2022-08-06 16:38:35.864138: W tensorflow/stream_executor/

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 852, in run
    _stream_output(self.process)
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 914, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 137

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 857, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/private/var/folders/zm/7yczzhb529xff1v0_ps_8k4h0000gp/T/tmpmz9312b0/docker-compose.y

In [ ]:
# Another way to measure performance

import multiprocessing


def invoke(endpoint_name):
    predictor = sagemaker.predictor.Predictor(
        endpoint_name,
        sm_sess,
        serializer=sagemaker.serializers.JSONSerializer(),
        deserializer=sagemaker.deserializers.BytesDeserializer(),
    )
    return predictor.predict(
        "{Bloomberg has decided to publish a new report on global economic situation.}"
    )


endpoint_name = predictor.endpoint_name
pool = multiprocessing.Pool(3)
results = pool.map(invoke, 3 * [endpoint_name])
pool.close()
pool.join()
print(results)